# Notebook Structure

### 1. read csv file as txt devided by comma "," by function `loadtxt()`
### 2. read csv file as txt line using `tf.decode_csv`


# 1. read csv file as txt devided by comma "," using function `loadtxt()`

In [1]:
import tensorflow as tf
import numpy as np

## sigmoid,hypothsis,cost
`sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))`

`hypothesis = tf.sigmoid(tf.matmul(X, W) + b)`

`cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))`

In [2]:
xy = np.loadtxt('data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 8])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([8, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)
# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # add this comment
    print("step\t cost")

    feed = {X: x_data, Y: y_data}
    for step in range(10001):
        sess.run(train, feed_dict=feed)
        if step % 200 == 0:
            print(step,"\t ", sess.run(cost, feed_dict=feed))

    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict=feed)
    print("\nHypothesis: ", h, "\nCorrect (Y): ", c, "\nAccuracy: ", a)

step	 cost
0 	  0.906505
200 	  0.683493
400 	  0.639451
600 	  0.619269
800 	  0.604048
1000 	  0.591115
1200 	  0.579872
1400 	  0.570037
1600 	  0.561403
1800 	  0.553794
2000 	  0.547064
2200 	  0.54109
2400 	  0.535765
2600 	  0.531002
2800 	  0.526725
3000 	  0.522872
3200 	  0.519388
3400 	  0.516229
3600 	  0.513354
3800 	  0.510731
4000 	  0.508331
4200 	  0.50613
4400 	  0.504105
4600 	  0.502239
4800 	  0.500515
5000 	  0.498919
5200 	  0.497439
5400 	  0.496064
5600 	  0.494784
5800 	  0.49359
6000 	  0.492476
6200 	  0.491435
6400 	  0.490459
6600 	  0.489545
6800 	  0.488687
7000 	  0.48788
7200 	  0.487122
7400 	  0.486408
7600 	  0.485735
7800 	  0.4851
8000 	  0.4845
8200 	  0.483934
8400 	  0.483398
8600 	  0.482891
8800 	  0.482411
9000 	  0.481956
9200 	  0.481524
9400 	  0.481114
9600 	  0.480725
9800 	  0.480356
10000 	  0.480004

Hypothesis:  [[ 0.374064  ]
 [ 0.91704977]
 [ 0.24144416]
 [ 0.9418987 ]
 [ 0.11297331]
 [ 0.78793555]
 [ 0.93945283]
 [ 0.59635198]
 [

# 2. read csv file as txt line using `tf.decode_csv`

1. use `tf.train.string_input_producer()` to create filename_queue
2. use `tf.TextLineReader()` to build a reader
3. use the `reader` to read `filename_queue` as `(key, value)`
4. we need to initialize the default value of records as `records_defaults`
5. then we can read the record by the `tf.decode_csv()` function
6. after that, we can slice the data as a batch
    
    - `# collect batches of csv in`
    - `# x_data = xy[:, 0:-1]`
    - `# y_data = xy[:, [-1]]`

### `train_x_batch, train_y_batch = tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)`


In [1]:
import tensorflow as tf
import numpy as np
filename_queue = tf.train.string_input_producer(
   ['data-03-diabetes.csv'], shuffle=False, name='filename_queue')

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# Default values, in case of empty columns. Also specifies the type of the decoded result.
record_defaults = [[0.], [0.], [0.], [0.], [0.], [0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

# collect batches of csv in

# x_data = xy[:, 0:-1]
# y_data = xy[:, [-1]]

# print(xy)

train_x_batch, train_y_batch = tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)


# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 8])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([8, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)
# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Start populating the filename queue.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])

    # add this comment
    print("step\t cost")
    
    feed = {X: x_batch, Y: y_batch}
    for step in range(10001):
        sess.run(train, feed_dict=feed)
        if step % 200 == 0:
            print(step, "\t ",sess.run(cost, feed_dict=feed))

    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict=feed)
    print("\nHypothesis:\n ", h, "\nCorrect (Y):\n ", c, "\nAccuracy\n: ", a)
    # Wait for threads to finish.    
    
coord.request_stop()
coord.join(threads)




step	 cost
0 	  0.550129
200 	  0.52761
400 	  0.509238
600 	  0.493854
800 	  0.480762
1000 	  0.469477
1200 	  0.459645
1400 	  0.450998
1600 	  0.443333
1800 	  0.436489
2000 	  0.43034
2200 	  0.424784
2400 	  0.419736
2600 	  0.415129
2800 	  0.410905
3000 	  0.407016
3200 	  0.403421
3400 	  0.400086
3600 	  0.396981
3800 	  0.39408
4000 	  0.391361
4200 	  0.388806
4400 	  0.386398
4600 	  0.384121
4800 	  0.381964
5000 	  0.379914
5200 	  0.377962
5400 	  0.376099
5600 	  0.374318
5800 	  0.37261
6000 	  0.370971
6200 	  0.369394
6400 	  0.367874
6600 	  0.366407
6800 	  0.36499
7000 	  0.363617
7200 	  0.362287
7400 	  0.360996
7600 	  0.359742
7800 	  0.358521
8000 	  0.357332
8200 	  0.356173
8400 	  0.355042
8600 	  0.353938
8800 	  0.352858
9000 	  0.351801
9200 	  0.350767
9400 	  0.349753
9600 	  0.348759
9800 	  0.347784
10000 	  0.346827

Hypothesis:
  [[ 0.17126067]
 [ 0.9255954 ]
 [ 0.04981807]
 [ 0.49506393]
 [ 0.00608971]
 [ 0.6701017 ]
 [ 0.66267228]
 [ 0.63820136